<a href="https://colab.research.google.com/github/Michael-Jimenez-C/Investigaci-n-de-operaciones/blob/main/Tarea3IO/Metodo_simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
class MetodoSimplex:
  #------------------------Método simplex base---------------------------
  def solve(M,sim,H):
    M=np.array(M,dtype="float64")
    ve=np.argmax(M[-1,:-1])
    vs=np.argmin(M[:-1,-1]/M[:-1,ve])
    H[vs]=sim[ve]
    M[vs]=M[vs]/M[vs][ve]
    
    for i in range(len(M)):
      if i!=vs:
        M[i]=M[i]*M[vs][ve]-M[vs]*M[i][ve]
    print(M,"\n")
    if len(M[-1,:-1][M[-1,:-1]>0])>0:
      return MetodoSimplex.solve(M,sim,H)
    
    H.append("Resultado")
    M[-1][-1]*=-1
    return {H[i]: M[i][-1] for i in range(len(H))}
  
  #------------------------Método dual---------------------------
  def __divsim(a,b):
    k=[]
    for i in range(len(a)):
      if (a[i]>0 and b[i]<0) or (a[i]<0 and b[i]>0):
        k.append(a[i]/b[i])
      else:
        k.append(-np.inf)
    return np.array(k)

  def dual(M,sim,H):
    M=np.array(M,dtype="float64")
    vs =np.argmin([i for i in M[:-1,-1]])
    ve=np.argmax(MetodoSimplex.__divsim(M[-1,:-1],M[vs,:-1]))

    H[vs]=sim[ve]
    M[vs]=M[vs]/M[vs][ve]
    for i in range(len(M)):
      if i!=vs:
        M[i]=M[i]*M[vs][ve]-M[vs]*M[i][ve]
    print(M)
    if len(M[:-1,-1][M[:-1,-1]<0])>0:
      return MetodoSimplex.dual(M,sim,H)
    H.append("Resultado")
    M[-1][-1]=abs(M[-1][-1])
    return {H[i]: M[i][-1] for i in range(len(H))}

In [ ]:
sim=['x','y','t','u','v']
H=['t','u','v']
M=[[2,1,1,0,0,180],
  [1,2,0,1,0,160],
  [1,1,0,0,1,100],
  [4,6,0,0,0,0]]
MetodoSimplex.solve(M,sim,H)

In [ ]:
sim=['x','y','t','u','v']
H=['t','u','v']
M=[[3,2,1,0,0,2400],
  [0,1,0,1,0,800],
  [2,0,0,0,1,1200],
  [5,2,0,0,0,0]]
MetodoSimplex.solve(M,sim,H)

#Min

In [ ]:
sim=['x','y','t','u','v']
H=['t','u','v']
M=[[-5,-15,1,0,0,-50],
    [-20,-5,0,1,0,-40],
    [15,2,0,0,1,60],
    [4,2,0,0,0,0]]
MetodoSimplex.dual(M,sim,H)